# 04 - Graph Neural Network Training

**TB Drug Discovery ML Pipeline - Phase 3**

This notebook covers:
1. Molecular graph representation
2. GNN model architectures (GCN, GAT, MPNN)
3. Training with early stopping
4. Ensemble with QSAR model

**Target:** Improve ROC-AUC over baseline QSAR

In [ ]:
# Imports
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Set seed
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
# Check PyTorch Geometric
try:
    import torch_geometric
    print(f"PyTorch Geometric: {torch_geometric.__version__}")
except ImportError:
    print("❌ PyTorch Geometric not installed")
    print("Install with: pip install torch-geometric")

## 1. Load Data

In [ ]:
# Load preprocessed data
data_path = Path.cwd().parent / "data" / "processed" / "cleaned_chembl_inhA.csv"

if data_path.exists():
    df = pd.read_csv(data_path)
    print(f"Loaded {len(df)} compounds")
    print(f"Active: {df['active'].sum()}, Inactive: {(~df['active'].astype(bool)).sum()}")
else:
    print("Run QSAR training first to generate cleaned data")

## 2. Create Molecular Graphs

In [ ]:
from src.gnn.featurizer import MolecularGraphFeaturizer, create_data_loaders

# Get SMILES and targets
smiles_list = df['smiles'].tolist()
targets = df['active'].tolist()

# Create DataLoaders
train_loader, val_loader, test_loader, featurizer = create_data_loaders(
    smiles_list=smiles_list,
    y_list=targets,
    train_ratio=0.8,
    val_ratio=0.1,
    batch_size=32,
    random_seed=SEED,
)

print(f"\nNode features: {featurizer.atom_dim}")
print(f"Edge features: {featurizer.bond_dim}")
print(f"\nTrain: {len(train_loader.dataset)}")
print(f"Val: {len(val_loader.dataset)}")
print(f"Test: {len(test_loader.dataset)}")

In [ ]:
# Visualize a sample graph
sample = train_loader.dataset[0]
print(f"Sample molecule: {sample.smiles}")
print(f"Nodes (atoms): {sample.num_nodes}")
print(f"Edges (bonds): {sample.edge_index.shape[1] // 2}")
print(f"Node features shape: {sample.x.shape}")
print(f"Target: {sample.y.item()}")

## 3. Train GNN Models

In [ ]:
from src.gnn.models import create_model
from src.gnn.trainer import GNNTrainer, EarlyStopping

# Model configuration
MODEL_TYPE = 'gat'  # Options: 'gcn', 'gat', 'mpnn', 'attentivefp'
HIDDEN_DIM = 128
NUM_LAYERS = 3
EPOCHS = 100
PATIENCE = 15

# Create model
model = create_model(
    model_type=MODEL_TYPE,
    node_dim=featurizer.atom_dim,
    edge_dim=featurizer.bond_dim,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    task='classification',
)

# Count parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model: {MODEL_TYPE.upper()}")
print(f"Parameters: {num_params:,}")

In [ ]:
# Create trainer
trainer = GNNTrainer(
    model=model,
    task='classification',
    learning_rate=1e-3,
)

# Early stopping
early_stopping = EarlyStopping(patience=PATIENCE, mode='min')

# Train
print("\nTraining...")
history = trainer.fit(
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=EPOCHS,
    early_stopping=early_stopping,
    checkpoint_dir='../models/gnn',
)

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Validation')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()

# ROC-AUC
train_auc = [m['roc_auc'] for m in history['train_metrics']]
val_auc = [m['roc_auc'] for m in history['val_metrics']]
axes[1].plot(train_auc, label='Train')
axes[1].plot(val_auc, label='Validation')
axes[1].axhline(y=0.75, color='r', linestyle='--', label='Target')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('ROC-AUC')
axes[1].set_title('ROC-AUC')
axes[1].legend()

plt.tight_layout()
plt.savefig('../results/figures/gnn_training_history.png', dpi=150)
plt.show()

## 4. Evaluate on Test Set

In [ ]:
# Evaluate
test_metrics = trainer.evaluate(test_loader)

print("\nTest Set Results:")
print("=" * 40)
print(f"  ROC-AUC:   {test_metrics['roc_auc']:.4f}")
print(f"  Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"  Precision: {test_metrics['precision']:.4f}")
print(f"  Recall:    {test_metrics['recall']:.4f}")
print(f"  F1-Score:  {test_metrics['f1']:.4f}")
print("=" * 40)

target_met = test_metrics['roc_auc'] >= 0.75
print(f"\nTarget (ROC-AUC >= 0.75): {'✅ PASSED' if target_met else '❌ NOT MET'}")

## 5. Compare with QSAR Baseline

In [ ]:
import json

# Load QSAR metrics
qsar_metrics_path = Path.cwd().parent / "models" / "qsar_metrics.json"

if qsar_metrics_path.exists():
    with open(qsar_metrics_path) as f:
        qsar_metrics = json.load(f)
    
    qsar_auc = qsar_metrics['test_metrics']['roc_auc']
    gnn_auc = test_metrics['roc_auc']
    
    print("\nModel Comparison:")
    print(f"  QSAR (Random Forest): {qsar_auc:.4f}")
    print(f"  GNN ({MODEL_TYPE.upper()}):         {gnn_auc:.4f}")
    print(f"  Difference:           {gnn_auc - qsar_auc:+.4f}")
else:
    print("QSAR metrics not found")

## 6. Ensemble Model

In [ ]:
from src.gnn.ensemble import EnsembleModel
from src.models import QSARModel
from src.data import DataPreprocessor, DescriptorCalculator

# Load QSAR components
qsar_model = QSARModel.load('../models/qsar_rf_model.pkl')
preprocessor = DataPreprocessor.load('../models/qsar_scaler.pkl')
calculator = DescriptorCalculator()

# Create ensemble
ensemble = EnsembleModel(
    qsar_model=qsar_model.model,
    gnn_model=model,
    featurizer=featurizer,
    preprocessor=preprocessor,
    strategy='weighted',
    task='classification',
)

print("Ensemble created")

In [ ]:
# Prepare test data for ensemble
test_smiles = [data.smiles for data in test_loader.dataset]
test_targets = np.array([data.y.item() for data in test_loader.dataset])

# Compute descriptors
df_test = pd.DataFrame({'smiles': test_smiles})
df_desc = calculator.calculate_from_dataframe(df_test, smiles_col='smiles')
X_test = df_desc[calculator.descriptor_names].values
X_test_scaled = preprocessor.transform(X_test)

# Optimize weights
best_weights, best_score = ensemble.optimize_weights(
    test_smiles, X_test_scaled, test_targets
)

print(f"\nOptimal weights: QSAR={best_weights[0]:.2f}, GNN={best_weights[1]:.2f}")
print(f"Best score: {best_score:.4f}")

In [ ]:
# Evaluate ensemble
ensemble_metrics = ensemble.evaluate(test_smiles, X_test_scaled, test_targets)

print("\nEnsemble Results:")
print("=" * 50)
print(f"  QSAR ROC-AUC:     {ensemble_metrics['qsar_roc_auc']:.4f}")
print(f"  GNN ROC-AUC:      {ensemble_metrics['gnn_roc_auc']:.4f}")
print(f"  Ensemble ROC-AUC: {ensemble_metrics['ensemble_roc_auc']:.4f}")
print("=" * 50)
print(f"  Improvement over QSAR: {ensemble_metrics['improvement_over_qsar']:+.4f}")
print(f"  Improvement over GNN:  {ensemble_metrics['improvement_over_gnn']:+.4f}")

## 7. Save Models

In [ ]:
# Save GNN model
gnn_path = Path.cwd().parent / "models" / "gnn"
gnn_path.mkdir(exist_ok=True)

trainer.save_model(str(gnn_path / f"{MODEL_TYPE}_model.pt"))
print(f"GNN saved: {gnn_path / f'{MODEL_TYPE}_model.pt'}")

# Save ensemble
ensemble.save(str(gnn_path / "ensemble"))
print(f"Ensemble saved: {gnn_path / 'ensemble'}")

# Save metrics
all_metrics = {
    'gnn_test_metrics': test_metrics,
    'ensemble_metrics': ensemble_metrics,
    'model_type': MODEL_TYPE,
    'optimal_weights': best_weights,
}

with open(gnn_path / 'gnn_metrics.json', 'w') as f:
    json.dump(all_metrics, f, indent=2, default=float)
print(f"Metrics saved: {gnn_path / 'gnn_metrics.json'}")

## Summary

### Results:

| Model | ROC-AUC | vs QSAR |
|-------|---------|----------|
| QSAR (RF) | See above | - |
| GNN | See above | See above |
| Ensemble | See above | See above |

### Key Findings:
- GNN learns directly from molecular graphs
- Ensemble combines strengths of both approaches
- Optimal weights balance descriptor-based and graph-based features

### Next Steps:
- Try different GNN architectures
- Hyperparameter tuning
- Interpretability analysis (attention weights)